In [5]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df_all_data = pd.read_csv('./dataset/storm_event_details_2010.csv')

for i in range(2011,2021):
    df_temp = pd.read_csv(f'./dataset/storm_event_details_{i}.csv')
    df_all_data = df_all_data.append(df_temp, ignore_index=True)


In [9]:
df_hur = df_all_data[df_all_data['EVENT_TYPE']=='Tornado']
df_hur = df_hur.drop(columns=['TOR_OTHER_WFO',
                                           'TOR_OTHER_CZ_STATE','TOR_OTHER_CZ_FIPS','TOR_OTHER_CZ_NAME','DATA_SOURCE','EPISODE_NARRATIVE',
                                            'EVENT_NARRATIVE','WFO','SOURCE','CZ_TIMEZONE','BEGIN_AZIMUTH','END_AZIMUTH','BEGIN_LAT',
                                            'END_LAT','BEGIN_LON','END_LON','STATE_FIPS','BEGIN_RANGE','END_RANGE','DAMAGE_CROPS',
                                            'BEGIN_TIME','END_TIME','BEGIN_LOCATION','END_LOCATION','FLOOD_CAUSE','MAGNITUDE_TYPE',
                                            'MAGNITUDE','CZ_FIPS','CZ_TYPE','CZ_NAME','CATEGORY'])
cols = ['INJURIES_INDIRECT', 'INJURIES_DIRECT', 'DEATHS_INDIRECT','DEATHS_DIRECT']
df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)
# cols = ['DEATHS_INDIRECT', 'DEATHS_DIRECT']
# df_hur = df_hur.assign(DEATHS_TOTAL=df_hur[cols].sum(1)).drop(cols,1)
# cols = ['DEATHS_INDIRECT', 'DEATHS_DIRECT']
# df_hur = df_hur.assign(DEATHS_TOTAL=df_hur[cols].sum(1)).drop(cols,1)
df_hur.info()
# df_hur = df_hur.dropna()
df_hur.head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14988 entries, 75 to 688592
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BEGIN_YEARMONTH  14988 non-null  int64  
 1   BEGIN_DAY        14988 non-null  int64  
 2   END_YEARMONTH    14988 non-null  int64  
 3   END_DAY          14988 non-null  int64  
 4   EPISODE_ID       14988 non-null  int64  
 5   EVENT_ID         14988 non-null  int64  
 6   STATE            14988 non-null  object 
 7   YEAR             14988 non-null  int64  
 8   MONTH_NAME       14988 non-null  object 
 9   EVENT_TYPE       14988 non-null  object 
 10  BEGIN_DATE_TIME  14988 non-null  object 
 11  END_DATE_TIME    14988 non-null  object 
 12  DAMAGE_PROPERTY  12945 non-null  object 
 13  TOR_F_SCALE      14988 non-null  object 
 14  TOR_LENGTH       14988 non-null  float64
 15  TOR_WIDTH        14988 non-null  float64
 16  HARM_TOTAL       14988 non-null  int64  
dtypes: float64

/tmp/ipykernel_2877/2998435838.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)


,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,EPISODE_ID,EVENT_ID,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,END_DATE_TIME,DAMAGE_PROPERTY,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,HARM_TOTAL
75,201011,30,201011,30,46405,269045,LOUISIANA,2010,November,Tornado,30-NOV-10 00:46:00,30-NOV-10 00:47:00,5.00K,EF0,1.26,25.0,0
304,201011,30,201011,30,46362,268819,SOUTH CAROLINA,2010,November,Tornado,30-NOV-10 19:45:00,30-NOV-10 19:49:00,1.50M,EF1,2.66,125.0,0
617,201010,18,201010,18,45236,262425,ARIZONA,2010,October,Tornado,18-OCT-10 16:43:00,18-OCT-10 16:47:00,10.00K,EF0,1.23,25.0,0
731,201011,29,201011,29,45412,263493,LOUISIANA,2010,November,Tornado,29-NOV-10 15:12:00,29-NOV-10 15:25:00,750.00K,EF4,13.96,400.0,0
732,201011,29,201011,29,45412,263494,LOUISIANA,2010,November,Tornado,29-NOV-10 16:18:00,29-NOV-10 16:26:00,10.00K,EF1,3.67,100.0,0
842,201001,20,201001,20,35166,203450,TEXAS,2010,January,Tornado,20-JAN-10 18:58:00,20-JAN-10 19:00:00,200.00K,EF0,1.48,75.0,0
871,201008,7,201008,7,41239,239944,NORTH DAKOTA,2010,August,Tornado,07-AUG-10 17:25:00,07-AUG-10 17:37:00,NaN,EF4,2.50,600.0,0
872,201008,7,201008,7,41241,239945,MINNESOTA,2010,August,Tornado,07-AUG-10 17:25:00,07-AUG-10 17:25:00,NaN,EF0,0.20,25.0,0
915,201001,20,201001,20,35166,203451,TEXAS,2010,January,Tornado,20-JAN-10 19:09:00,20-JAN-10 19:10:00,40.00K,EF1,0.47,75.0,0
929,201007,17,201007,17,40144,263432,MINNESOTA,2010,July,Tornado,17-JUL-10 16:12:00,17-JUL-10 16:13:00,NaN,EF0,0.39,15.0,0


In [11]:
df_hur = df_hur.sort_values('HARM_TOTAL', ascending=False)
df_hur.head(20)

,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,EPISODE_ID,EVENT_ID,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,END_DATE_TIME,DAMAGE_PROPERTY,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,HARM_TOTAL
100619,201105,22,201105,22,49972,296617,MISSOURI,2011,May,Tornado,22-MAY-11 16:40:00,22-MAY-11 17:00:00,2.80B,EF5,8.80,1600.0,1311
134758,201104,27,201104,27,50455,314662,ALABAMA,2011,April,Tornado,27-APR-11 15:48:00,27-APR-11 16:35:00,1.50B,EF4,41.79,1760.0,852
136287,201104,27,201104,27,50455,314663,ALABAMA,2011,April,Tornado,27-APR-11 16:35:00,27-APR-11 17:14:00,700.00M,EF4,32.53,2600.0,720
344213,201512,26,201512,26,101364,606471,TEXAS,2015,December,Tornado,26-DEC-15 18:46:00,26-DEC-15 18:59:00,26.00M,EF4,9.21,550.0,478
265662,201305,20,201305,20,74673,451572,OKLAHOMA,2013,May,Tornado,20-MAY-13 14:04:00,20-MAY-13 14:35:00,2.00B,EF5,12.00,1900.0,231
75537,201104,27,201104,27,49915,300457,TENNESSEE,2011,April,Tornado,27-APR-11 19:34:00,27-APR-11 19:56:00,23.00M,EF4,20.40,800.0,204
122118,201106,1,201106,1,52130,323440,MASSACHUSETTS,2011,June,Tornado,01-JUN-11 15:17:00,01-JUN-11 16:09:00,227.60M,EF3,31.52,880.0,203
312392,201404,27,201404,27,83594,504760,ARKANSAS,2014,April,Tornado,27-APR-14 18:25:00,27-APR-14 18:59:00,210.00M,EF4,24.24,1320.0,179
632636,202003,3,202003,3,146901,882999,TENNESSEE,2020,March,Tornado,03-MAR-20 00:32:00,03-MAR-20 00:54:00,1.09B,EF3,21.49,1150.0,172
599455,201905,27,201905,27,136052,816403,OHIO,2019,May,Tornado,27-MAY-19 21:41:00,27-MAY-19 22:13:00,500.00M,EF4,18.17,1050.0,167
